# Load

## Import

In [ ]:
%reload_ext autoreload
%autoreload 2
%reload_ext cython
%reload_ext line_profiler
import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..', 'module'))
import wiki
import dill
import scipy as sp
import numpy as np
import pandas as pd
import networkx as nx

In [ ]:
from ipywidgets import interact, widgets, Layout
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff

## Networks

In [ ]:
topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
          'genetics', 'immunology', 'molecular biology', 'chemistry', 'biophysics',
          'energy', 'optics', 'earth science', 'geology', 'meteorology',
          'philosophy of language', 'philosophy of law', 'philosophy of mind',
          'philosophy of science', 'economics', 'accounting', 'education',
          'linguistics', 'law', 'psychology', 'sociology', 'electronics',
          'software engineering', 'robotics',
          'calculus', 'geometry', 'abstract algebra',
          'Boolean algebra', 'commutative algebra', 'group theory', 'linear algebra',
          'number theory', 'dynamical systems and differential equations']

In [ ]:
import wiki

path_saved = '/Users/harangju/Developer/data/wiki/graphs/dated'

networks = {}
for topic in topics:
    print(topic, end=' ')
    networks[topic] = wiki.Net(path_graph=os.path.join(path_saved, topic + '.pickle'),
                               path_barcodes=os.path.join(path_saved, topic + '.barcode'))

## Models

In [ ]:
simulation = '20200422_1318'

In [ ]:
base_dir = os.path.join('/', 'Users', 'harangju', 'Developer', 'data', 'wiki', 'simulations')
session_dir = os.path.join(base_dir, simulation)

In [ ]:
filenames = sorted(os.listdir(session_dir))
filenames[:3]

In [ ]:
model_topics = list(set([filename.split('_')[1] for filename in filenames[:-1]]))
model_topics[:3]

In [ ]:
# models = {topic: [dill.load(open(os.path.join(session_dir, filename), 'rb'))
#                   for filename in filenames
#                   if filename.split('_')[1]==topic]
#                for topic in model_topics}
model_paths = {topic: [os.path.join(session_dir, filename)
                       for filename in filenames[:-1]
                       if filename.split('_')[1]==topic]
               for topic in model_topics}

In [ ]:
{topic: model_paths[topic] for topic in model_topics[:3]}

# Analysis

## Basic network statistics

In [ ]:
import bct
import pickle
from networkx.algorithms.cluster import clustering
from networkx.algorithms import betweenness_centrality
from networkx.convert_matrix import to_numpy_array

### Real

In [ ]:
import pickle
import pandas as pd
path_analysis = '/Users/harangju/Developer/data/wiki/analysis/'
df = pickle.load(open(path_analysis+'stats.pickle', 'rb'))
df_expand = pickle.load(open(path_analysis+'stats_expand.pickle', 'rb'))
df.topic = df.topic.astype('object')
df.measure = df.measure.astype('object')
df_expand.topic = df_expand.topic.astype('object')
df_expand.measure = df_expand.measure.astype('object')
df_mean = df_expand\
    .groupby(['topic', 'measure'], as_index=False)\
    .mean()\
    .pivot(index='topic', columns='measure', values='value')\
    .reset_index()

In [ ]:
df

### Model

In [ ]:
measures = {'indegree': lambda g: [x[1] for x in g.in_degree],
            'outdegree': lambda g: [x[1] for x in g.out_degree],
            'clustering': lambda g: list(clustering(g).values()),
            'centrality': lambda g: list(betweenness_centrality(g).values()),
            'modularity': lambda g: g.graph['modularity'],
            'coreness': lambda g: g.graph['coreness_be']}

In [ ]:
df_model = pd.DataFrame()
for topic in model_paths.keys():
    print(topic)
    network = networks[topic]
    for i, model_path in enumerate(model_paths[topic]):
        print(i, end=' ')
        model = dill.load(open(model_path, 'rb'))
        df_model = pd.concat(
            [df_model] + [pd.DataFrame([[topic, i, measure, func(model.graph)]],
                                columns=['topic','model','measure','value'])
                    for measure, func in measures.items()],
            ignore_index=True)
    print('')

In [ ]:
df_model

In [ ]:
df_model_expand = df_model\
    .drop('model', axis=1)\
    .value\
    .apply(pd.Series)\
    .merge(df_model, left_index=True, right_index=True)\
    .drop('value', axis=1)\
    .melt(id_vars=['topic','measure'])\
    .drop('variable', axis=1)\
    .dropna()
df_model_expand

In [ ]:
df_model_mean = df_model_expand\
    .groupby(['topic', 'measure'], as_index=False)\
    .mean()\
    .pivot(index='topic', columns='measure', values='value')
df_model_mean

### Save

In [ ]:
path_analysis = '/Users/harangju/Developer/data/wiki/analysis/'
pickle.dump(df_model, open(os.path.join(path_analysis,'stats_model.pickle'),'wb'))
# pickle.dump(df_model_expand, open(os.path.join(path_analysis,'stats_model_expand.pickle'),'wb'))
# pickle.dump(df_model_mean, open(os.path.join(path_analysis,'stats_model_mean.pickle'),'wb'))

## Persistent homology

# Plot

In [ ]:
import plotly.subplots as sb

In [ ]:
import plotly.express as px
fig = px.colors.qualitative.swatches()
# fig.show()

In [ ]:
first_n_nodes = 10

## Growth

In [ ]:
save_fig = True
path_result = os.path.join('/','Users','harangju','Box Sync','Research','my papers','wikipedia','results')
path_plot = '3 model growth'

In [ ]:
for topic in model_paths.keys():
    fig = sb.make_subplots(1,2)
    network = networks[topic]
    fig.add_trace(go.Histogram(x=[d for _,d in network.graph.degree], nbinsx=30, 
                               name='empirical'),
                  row=1, col=1)
    fig.update_yaxes(title_text='number of edges', row=1, col=1)
    fig.update_xaxes(title_text='degree', row=1, col=1)
    fig.add_trace(go.Scatter(x=sorted([network.graph.nodes[n]['year'] 
                                       for n in network.graph.nodes]),
                             y=list(range(1,len(network.graph.nodes)+1)),
                             mode='lines', name='empirical', showlegend=False,
                             line={'color': px.colors.qualitative.Plotly[0]}),
                  row=1, col=2)
    fig.update_yaxes(title_text='number of nodes', row=1, col=2)
    fig.update_xaxes(title_text='year', row=1, col=2)
    fig.update_layout(title=topic, template='plotly_white')
    for i, model_path in enumerate(model_paths[topic]):
        model = dill.load(open(model_path, 'rb'))
        fig.add_trace(go.Histogram(x=[d for _,d in model.graph.degree], nbinsx=30, 
                                   name=f"model {i}", marker_color=px.colors.qualitative.Plotly[1+i]),
                      row=1, col=1)
        fig.add_trace(go.Scatter(x=sorted([model.graph.nodes[n]['year'] for n in model.graph.nodes]),
                                 y=list(range(1,len(model.graph.nodes)+1)),
                                 mode='lines', name=f"model {i}", showlegend=False,
                                 line={'color': px.colors.qualitative.Plotly[1+i]}),
                      row=1, col=2)
    start_date = sorted(list(nx.get_node_attributes(model.graph_parent, 'year').values()))[first_n_nodes]
    fig.add_trace(go.Scatter(x=[start_date, start_date], y=[0, max(len(model.graph.nodes),
                                                                   len(network.graph.nodes))],
                             mode='lines', name='model start',
                             line={'color': 'black',
                                   'dash': 'dash'}),
                  row=1, col=2)
    fig.show()
    if save_fig:
        fig.write_image(os.path.join(path_result, path_plot, topic+'.pdf'))

## Static

In [ ]:
path_plot = '3 model network_stats'

In [ ]:
df_mean = df_mean.sort_values('topic', ascending=True, ignore_index=True)
df_model_mean = df_model_mean.sort_values('topic', ascending=True, ignore_index=True)

In [ ]:
ranges = {'clustering': [0,0.3],
          'centrality': [0,0.04],
          'indegree': [0,10],
          'outdegree': [0,10],
          'coreness': [0,1],
          'modularity': [0,1]}
dticks = {'clustering': 0.1,
          'centrality': 0.01,
          'indegree': 1,
          'outdegree': 1,
          'coreness': .5,
          'modularity': .5}

In [ ]:
for measure in ['clustering','centrality','indegree','outdegree','coreness','modularity']:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_mean[measure],
                             y=df_model_mean[measure],
                             mode='markers',
                             marker={'color': '#2A3F5F'},
                             hovertext=df_mean.topic,
                             showlegend=False))
    fig.add_trace(go.Scatter(x=ranges[measure], y=ranges[measure],
                             mode='lines',
                             line={'dash': 'dash',
                                   'color': '#2A3F5F'},
                             showlegend=False))
    fig.update_layout(template='plotly_white',
                      width=500, height=500,
                      title=measure,
                      xaxis={'title': 'real',
                             'range': ranges[measure],
                             'dtick': dticks[measure]},
                      yaxis={'title': 'model',
                             'range': ranges[measure],
                             'dtick': dticks[measure]})
    fig.show()
#     if save_fig:
#         fig.write_image(os.path.join(path_fig,path_plot,f"summary_{measure}.pdf"))